In [ ]:
# default_exp input_file

# Input file

> Convenience function to construct the input file required by py2gift.

In [ ]:
# export

import pathlib
from typing import Union, Optional, Callable

import py2gift.util

import yaml

In [ ]:
test_file = '_input_.yaml'

# Reading

Code to parse input files.

### `extract_class_settings`

In [ ]:
# export

def extract_class_settings(category_name: Union[str, list], class_name: str, settings: dict):
    
    category_found = False
    
    for cat in settings['categories']:
        
        if cat['name'] == category_name:
            
            category_found = True
            
            for cls in cat['classes']:
                
                if cls['name'] == class_name:
                    
                    return cls
    
    else:
        
        if category_found:
            
            print(f'cannot find the requested class, {class_name}')
            sys.exit(1)
        
        else:
            
            print(f'cannot find the requested category, {category_name}')
            sys.exit(1)

In [ ]:
%%writefile {test_file}

output file: third_midterm.yaml
pictures base directory: tc/midterm3
path to gift-wrapper: '~/gift-wrapper/wrap.py'

categories:

  - name: Entropy of the input given the output

    classes:

      - name: EntropyOfInputGivenOutput

        question base name: Entropy of input given the output
        
        init parameters:
            
            picture_file: DMCs/entropy_at_input_given_output.tex

        statement: |
          Consider...
          

        feedback: |
          One way of tackling the problem is...
          

        number of instances: 2


In [ ]:
with open(test_file) as yaml_data:

    settings = yaml.load(yaml_data, Loader=yaml.FullLoader)

extract_class_settings('Entropy of the input given the output', 'EntropyOfInputGivenOutput', settings)

# In-memory *input file*

In [ ]:
# export

def initialize(output_file: str, pictures_directory: str, ) -> dict:
    
    settings = {}
    
    settings['output file'] = output_file
    settings['pictures base directory'] = pictures_directory
    settings['path to gift-wrapper'] = '~/gift-wrapper/wrap.py'
    settings['categories'] = None
    
    return settings

In [ ]:
settings_dict = initialize(output_file='quiz.yaml', pictures_directory='quiz/pics')

In [ ]:
# export

def set_class_preamble(settings: dict, category_name: str, base_category: Optional[str] = None, test_mode: bool = False) -> Union[str, list]:
    
    if test_mode:

        category_name = 'test'

    else:

        if base_category:

            category_name = [base_category, f'{base_category}/{category_name}']
    
    
    if settings['categories'] is None:
        
        settings['categories'] = []
        
    settings['categories'].append({'name': category_name, 'classes': None})
    
    return category_name

In [ ]:
set_class_preamble(settings_dict, 'Test category')
settings_dict

In [ ]:
# export

def set_class_closing(settings: dict, n_instances: int, time: Optional[int] = None) -> None:
    
    if settings['categories'][-1]['classes'] is None:
        
        settings['categories'][-1]['classes'] = [{}]
    
    settings['categories'][-1]['classes'][-1]['number of instances'] = n_instances
    settings['categories'][-1]['classes'][-1]['time'] = time

In [ ]:
set_class_closing(settings_dict, 2, 15)
settings_dict

# Writing

Code to write input files.

## Header

A function to write the header of the file.
* `file`: name of the input file to be created

In [ ]:
# export

def write_header(file: Union[str, pathlib.Path], output_file: str, pictures_directory: str, ) -> None:
    
    settings = initialize(output_file, pictures_directory)
    
    py2gift.util.dict_to_yaml(settings, file)
    
#     with open(file, 'w') as f:
        
#         f.write(f'output file: {output_file}\n')
#         f.write(f'pictures base directory: {pictures_directory}\n')
#         f.write("path to gift-wrapper: '~/gift-wrapper/wrap.py'\n")
#         f.write('\n')
        
#         f.write('categories:\n')

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')

In [ ]:
%cat {test_file}

## Class

### `write_class_preamble`

A function to write the preamble for a class which includes the name of the category. Strictly speaking, it's not necessary to create a new category every time a class is added, but this is more general and a category can show up many times in the input file.
* `file`: name of the input file to be *appended to*
* `category_name`: name of the category in which the class will be encompassed
* `base_category`: a parent category for `category_name` (optional)
* `test_mode`: if `True`, the last two parameters are overriden and the category is simply called `test`

For convenience, the final `category_name` (notice that it might get tweaked because of the other parameters) is returned.

In [ ]:
# export

def write_class_preamble(file: Union[str, pathlib.Path], category_name: str, base_category: Optional[str] = None, test_mode: bool = False) -> Union[str, list]:
    
    
    settings = py2gift.util.yaml_to_dict(file)
    category_name = set_class_preamble(settings, category_name, base_category, test_mode)
    py2gift.util.dict_to_yaml(settings, file)

#     # file is *appended* (not overwritten)
#     with open(file, 'a') as f:

#         f.write('\n')
        
#         if test_mode:
            
#             category_name = 'test'
        
#         else:
        
#             if base_category:

#                 category_name = [base_category, f'{base_category}/{category_name}']
        
#         f.write(f'  - name: {category_name}\n\n')
#         f.write('    classes:\n')
    
    return category_name

In [ ]:
write_class_preamble(test_file, 'Test category')

In [ ]:
%cat {test_file}

Parameter `base_category` allows to create hierarchical categories

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')
write_class_preamble(test_file, 'Test category', base_category='base')

In [ ]:
%cat {test_file}

In `test_mode`

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')
write_class_preamble(test_file, 'Test category', base_category='base', test_mode=True)

In [ ]:
%cat {test_file}

### `write_class_closing`

A function to write the closing settings of a class.
* `file`: name of the input file to be *appended to*
* `n_instances`: number of questions of this class that will be created (the `setup` method of the class should be non-deterministic)
* `time`: time in minutes (an *integer*) estimated necessary to solve the question

In [ ]:
# export

def write_class_closing(file: Union[str, pathlib.Path], n_instances: int, time: Optional[int] = None) -> None:
    
    settings = py2gift.util.yaml_to_dict(file)
    category_name = set_class_closing(settings, n_instances, time)
    py2gift.util.dict_to_yaml(settings, file)

#     with open(file, 'a') as f:

#         f.write(f'\n        number of instances: {n_instances}')
        
#         if time:
            
#             f.write(f'\n        time: {time}')

In [ ]:
write_class_closing(test_file, 2, 15)

In [ ]:
%cat {test_file}

In [ ]:
!rm {test_file}

In [ ]:
def write_class_closing2(file: Union[str, pathlib.Path], n_instances: int, time: Optional[int] = None) -> None:
    
    settings = py2gift.util.yaml_to_dict(file)
    category_name = set_class_closing(settings, n_instances, time)
    py2gift.util.dict_to_yaml(settings, file)

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')
write_class_preamble(test_file, 'Test category', base_category='base', test_mode=True)
write_class_closing2(test_file, 2, 15)

In [ ]:
%cat {test_file}

# Categories

In [ ]:
# export

def function_to_make_hierarchical_category_name(base_category: str) -> Callable[[str], list]:
    
    def make_subcategory(category: str) -> list:
    
        return [base_category, f'{base_category}/{category}']
    
    return make_subcategory

In [ ]:
f = function_to_make_hierarchical_category_name('2020 exam')
f('Random')